In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
from utils.reparam_module import ReparamModule

In [63]:
data = torch.load('./pattern-toys.pth')
print(len(data))

11268


In [64]:
# Example to show how to find frequent sequential patterns
# from a given sequence database subject to constraints
from sequential.seq2pat import Seq2Pat, Attribute

# data = [
#     [1,2,112,2223,3,134],
#     [1,2,6456,324,1521345,123],
#     [1,2,125,4365,23421,203985],
#     [2,3,12358,4532,23541,38954],
#     [2,3,353,4234,45453,4678],
#     [2,3,234,3342,11234,465],
# ]

# Seq2Pat over 3 sequences
seq2pat = Seq2Pat(sequences=data, n_jobs=2, max_span=10)
# Patterns that occur at least twice (A-D)
patterns = seq2pat.get_patterns(min_frequency=2)
patterns_value = [_[:-1] for _ in patterns]
patterns_count = [_[-1] for _ in patterns]
# print(patterns)
print(len(patterns))

23426


In [65]:
test = [torch.tensor(_) for _ in patterns_value]
test = pad_sequence(test, batch_first=True)
seq_len = test.bool().sum(-1)

In [66]:
new_patterns = deepcopy(patterns)
for idx, _ in enumerate(test):
    last_item = _[seq_len[idx] - 1]
    succesor_list = torch.where(test[:, 0] == last_item)[0]
    for succesor in succesor_list:
        count = min(patterns_count[succesor], patterns_count[idx])
        if count > 3:
            new_patterns += [
                patterns_value[idx] + \
                patterns_value[succesor][1:] + \
                [count]
            ]
print(len(new_patterns))


30288


In [67]:
patterns = new_patterns

In [68]:
original_train = torch.load('./dataset/amazon-toys-seq/toy/train_ori.pth')
len(original_train)

61616

In [69]:
max_seq_len = 20
def truncate_or_pad(seq):
    cur_seq_len = len(seq)
    if cur_seq_len > max_seq_len:
        return seq[-max_seq_len:]
    else:
        return seq + [0] * (max_seq_len - cur_seq_len)


train_set = set()

for _ in original_train:
    train_set.add(tuple(_[1] + [_[2]]))

for pattern in patterns:
    seq = pattern[:-1]
    seq_len = len(seq)
    for _ in range(1, seq_len):
        train_set.add(tuple(
            truncate_or_pad(seq[:_]) + [seq[_]],
        ))

In [70]:
train_list = []
for _ in train_set:
    train_list.append([
        0,
        list(_)[:-1],
        list(_)[-1],
        sum([a != 0 for a in list(_)[:-1]]),
        1,
        0
    ])
print(len(train_list))
print(len(original_train))

88344
61616


In [71]:
torch.save(train_list + original_train, 'train_new-high3.pth')

In [34]:
torch.save(train_list, 'train_new-filter.pth')